# run_model_toy06

From https://github.com/tardis-sn/tardis-setups/tree/master/rad_trans_models with some changes.

---

In [ ]:
EPOCH = '20d'
ATOM_DATA = 'kurucz_cd23_latest.h5'             # in ~/Downloads/tardis-data
PICKLE_DIR = '/storage/epassaro/Toy_06/kurucz'  # where to dump pickled sims
NTHREADS = 32

In [ ]:
import os
import re
import yaml
import numpy as np
import pandas as pd
from astropy import units as u
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.util.base import parse_quantity

In [ ]:
os.makedirs(PICKLE_DIR, exist_ok=True)

In [ ]:
lbols = np.array([3.05e+42, 8.91e+42, 1.10e+43, 1.00e+43])*u.erg/u.s
final_params = {'5d':  (5*u.d, lbols[0],  20500.*u.km/u.s), 
                '10d': (10*u.d, lbols[1], 17000.*u.km/u.s), 
                '15d': (15*u.d, lbols[2], 10000.*u.km/u.s), 
                '20d': (20*u.d, lbols[3],  5500.*u.km/u.s),}

In [ ]:
pattern_remove_bracket = re.compile('\[.+\]')
t0_pattern = re.compile('tend = (.+)\n')

In [ ]:
def read_blondin_toymodel(fname):

    with open(fname, 'r') as f:
        for line in f:
            if line.startswith("#idx"):
                break
        else:
            raise ValueError('File {0} does not conform to Toy Model format as it does not contain #idx')
            
    columns = [pattern_remove_bracket.sub('', item) for item in line[1:].split()]

    raw_blondin_csv =  pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, names=columns)
    raw_blondin_csv.set_index('idx', inplace=True)

    blondin_csv = raw_blondin_csv.loc[:, ['vel', 'dens', 'temp', 'X_56Ni0', 'X_Ti', 'X_Ca', 'X_S', 'X_Si', 'X_O', 'X_C']]
    rename_col_dict = {'vel': 'velocity', 'dens': 'density', 'temp': 't_rad'}
    rename_col_dict.update({item:item[2:] for item in blondin_csv.columns[3:]})
    rename_col_dict['X_56Ni0'] = 'Ni56'
    blondin_csv.rename(columns=rename_col_dict, inplace=True)
    blondin_csv.iloc[:, 3:] = blondin_csv.iloc[:,3:].divide(blondin_csv.iloc[:,3:].sum(axis=1), axis=0)

    # changing velocities to outer boundary
    new_velocities = 0.5 * (blondin_csv.velocity.iloc[:-1].values + blondin_csv.velocity.iloc[1:].values)
    new_velocities = np.hstack((new_velocities, [2 * new_velocities[-1] - new_velocities[-2]]))
    blondin_csv['velocity'] = new_velocities

    with open(fname, 'r') as f:
        t0_string = t0_pattern.findall(f.read())[0]

    t0 = parse_quantity(t0_string.replace('DAYS', 'day'))
    
    blondin_dict = {}
    blondin_dict['model_density_time_0'] = str(t0)
    blondin_dict['description'] = 'Converted {0} to csvy format'.format(fname)
    blondin_dict['tardis_model_config_version'] = 'v1.0'
    blondin_dict_fields = [dict(name='velocity', unit='km/s', desc='velocities of shell outer bounderies.')]
    blondin_dict_fields.append(dict(name='density', unit='g/cm^3', desc='mean density of shell.'))
    blondin_dict_fields.append(dict(name='t_rad', unit='K', desc='radiative temperature.'))

    for abund in blondin_csv.columns[3:]:
        blondin_dict_fields.append(dict(name=abund, desc='Fraction {0} abundance'.format(abund)))
    blondin_dict['datatype'] = {'fields':blondin_dict_fields}

    return blondin_dict, blondin_csv

In [ ]:
def run_final_models_plus_pickle(params, fname, atom_data, nthreads, pickle_dir):
    
    model_config = Configuration.from_yaml(fname)
    model_config.montecarlo.nthreads = nthreads
    model_config.atom_data = atom_data
    model_config.model.v_inner_boundary = params[2]
    model_config.model.v_outer_boundary = 35000*u.km/u.s
    model_config.supernova.luminosity_requested = params[1]
    model_config.supernova.time_explosion = params[0]
    
    sim = Simulation.from_config(model_config)
    sim.run()
    
    import pickle

    dump = '{}/toy06_t{}_v{}.pickle'.format(pickle_dir, params[0].value, params[2].value)
    
    with open(dump, 'wb') as dumpfile:
        pickle.dump(sim, dumpfile, protocol=0)
    
    return

In [ ]:
blondin_dict, blondin_csv = read_blondin_toymodel('snia_toy06.dat')

In [ ]:
blondin_dict['v_inner_boundary'] = '9000 km/s'
blondin_dict['v_outer_boundary'] = '35000 km/s'
blondin_dict['model_isotope_time_0'] = '0. d'
csvy_file = '---\n{0}\n---\n{1}'.format(yaml.dump(blondin_dict, default_flow_style=False), 
                                        blondin_csv.to_csv(index=False))

In [ ]:
with open('blondin_compare_06.csvy', 'w') as f:
    f.write(csvy_file)

In [ ]:
epochs = np.array([5,10,15,20])*u.d
velocity_grid = np.arange(10000, 26500, 500)*u.km/u.s

In [ ]:
model_grid = np.array(np.empty((epochs.shape[0]*velocity_grid.shape[0], 3)))
model_grid = []

for i, epoch in enumerate(epochs):
    for j, velocity in enumerate(velocity_grid):
        model_grid.append((epoch, lbols[i], velocity-2000*u.km/u.s*i))
        
print(len(model_grid))

In [ ]:
run_final_models_plus_pickle(final_params[EPOCH], 'blondin_model_compare_06.yml', 
                             ATOM_DATA, NTHREADS, PICKLE_DIR)